In [2]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import os

from slide_util import slide_to_tiles

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.metrics import precision_score, recall_score, roc_auc_score

In [5]:
for slide_image in pd.read_csv("C:\\Code\\DL\\bbosis\\data\\test_set.csv")['img_pth']:
    bag = slide_to_tiles(np.asarray(ImageOps.exif_transpose(Image.open(f"C:\\Code\\DL\\bbosis\\data\\Training data\\{slide_image}.jpg")).convert("RGB")), 224, 0, -1)
    for i, tile in enumerate(bag):
        # save tile as jpg in a folder with the image id
        os.makedirs(f"C:\\Code\\DL\\bbosis\\data\\bags\\test\\{slide_image}", exist_ok=True)
        tile_image = Image.fromarray(tile)
        tile_image.save(f"C:\\Code\\DL\\bbosis\\data\\bags\\test\\{slide_image}\\{slide_image}_{i}.jpg")


In [6]:
for slide_image in pd.read_csv("C:\\Code\\DL\\bbosis\\data\\train_set.csv")['img_pth']:
    bag = slide_to_tiles(np.asarray(ImageOps.exif_transpose(Image.open(f"C:\\Code\\DL\\bbosis\\data\\Training data\\{slide_image}.jpg")).convert("RGB")), 224, 0, -1)
    for i, tile in enumerate(bag):
        # save tile as jpg in a folder with the image id
        os.makedirs(f"C:\\Code\\DL\\bbosis\\data\\bags\\train\\{slide_image}", exist_ok=True)
        tile_image = Image.fromarray(tile)
        tile_image.save(f"C:\\Code\\DL\\bbosis\\data\\bags\\train\\{slide_image}\\{slide_image}_{i}.jpg")

In [ ]:
class ImageBagDataset(Dataset):
    def __init__(self, csv_file, train=True):
        self.data = pd.read_csv(csv_file)
        if train:
            self.path = "C:\\Code\\DL\\bbosis\\data\\bags\\train\\"
        else:
            self.path = "C:\\Code\\DL\\bbosis\\data\\bags\\test\\"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        bag_dir = os.path.join(self.path, row['img_pth'])
        bag = []
        for img_name in os.listdir(bag_dir):
            img_path = os.path.join(bag_dir, img_name)
            img = Image.open(img_path).convert("RGB")
            img_array = np.array(img)
            bag.append(img_array)
        label = row['label']
        id_ = row['img_pth']
        bag_tensor = torch.tensor(bag).permute(0,3,1,2).float()  # [K, 3, 224, 224]
        return bag_tensor, label, id_